In [1]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
output_notebook()


Loading BokehJS ...

In [6]:
from pydgrid import pydgrid

In [8]:
sys1 = pydgrid.pydgrid()
sys1.read('lv_europe_connected.json')  # Load data
sys1.read_loads_shapes('load_shapes_list.json')

TypeError: 'module' object is not callable

## CIGRE LV European System

In [ ]:
sys1.pf()                            # execute power flow 

In [ ]:
pydss.set_load_factor(15*60*60,sys1.params_pf,sys1.params_lshapes)
pydss.pf_eval(sys1.params_pf);

In [ ]:
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

In [ ]:
%timeit sys1.bokeh_tools()


### Graph with obtained results

In [ ]:

p = figure(width=800, height=800,
           title='European LV Network (CIGRE)')#,   x_range=[-200,200], y_range=[-350, 50])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size='s_radio', color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

### Interaction with powers

In [ ]:
s_0_3pn = sys1.pq_3p # 
s_0_1p  = sys1.pq_1p  # 

p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           x_range = [0,200], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([0,300],[231*1.05,231*1.05], color='red', line_width=5)
p.line([0,300],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_loads(load_factor=1.0):

    sys1.pq_1p = load_factor*s_0_1p
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [ ]:
from ipywidgets import interact
interact(update_loads, load_factor=(-1,1.2, 0.01))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

In [ ]:
df['nodes'] = sys1.nodes
df['I_node_m'] = np.abs(sys1.I_node)
df['V_node_m'] = np.abs(sys1.V_node)
df['phi'] = np.angle(sys1.I_node, deg=True) - np.angle(sys1.V_node, deg=True)
df['s'] = np.conjugate(sys1.I_node)*sys1.V_node
df['p'] = df['s'].real
df['q'] = df['s'].imag

In [ ]:
self = sys1

node_sorter = []
for bus in self.buses:
    for node in range(10):
        bus_node = '{:s}.{:s}'.format(str(bus['bus']),str(node))
        if bus_node in self.nodes:
            node_idx = self.nodes.index(bus_node) 
            node_sorter += [node_idx]
                
                

In [ ]:
sys1.V_node[node_sorter]